<a href="https://colab.research.google.com/github/shivam-sunita-puri/TIYA/blob/main/Calculate_average_heart_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
########################################################
# Commands that must be run before using the functions
########################################################

# Substitute YYY below with token received
access_token='YYY'
it will say
# Change date below to correct date
url = "https://api.fitbit.com/1/user/-/activities/heart/date/2024-11-05/1d/1sec.json"  # Replace date as needed

# import packages
import pprint
import requests
import time
from datetime import datetime, timedelta
import pandas as pd

# set correct timezone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Australia/Melbourne /etc/localtime
!date

########################################################
#
# calculate_average_heart_rate(start_time, end_time)
# ==================================================
#
# GOAL:
#
# - Calculate average heart rate for the period specified
#
# USAGE:
#
# - Call the function below with start_time and end_time given as strings in HH:MM:SS format (24 hours notation)
#
# - If all the data in the time window is already on the FitBit website, the function returns the avergae heart rate in the time window
#
# - If some of the data still weren't uploaded to FitBit website (data is uploaded every 15-20 minutes), there will be an error message and the function will be returning -99
#
###########################################################
def calculate_average_heart_rate(start_time, end_time):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objects
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    parsed_start_time = datetime.strptime(start_time, time_format)
    parsed_end_time = datetime.strptime(end_time, time_format)

    # Compare the dates
    if parsed_start_time < df['parsed_time'][0]:
        print(f"start time {parsed_start_time} is earlier than the time stamp of the first data sample {df['parsed_time'].iloc[0]}")
        return -99
    elif parsed_end_time > df['parsed_time'].iloc[-1]:
        print(f"end time {parsed_end_time} is later than the time stamp of the last data sample {df['parsed_time'].iloc[-1]}")
        return -99


    times = [row['time'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {len(times)} time stamps averaged:")
    print(f"DEBUG INFORMATION: {times}")

    values_to_average = [row['value'] for index, row in df.iterrows() if row['parsed_time'] >= parsed_start_time and row['parsed_time'] <= parsed_end_time]
    print(f"DEBUG INFORMATION: {values_to_average}")

    return sum(values_to_average) / len(values_to_average)

  else:
    pprint.pprint(data)

########################################################
#
# average_heart_rate, first_sample_time, last_sample_time = average_last_X_minutes_available(minutes)
# =================================================================
#
# GOAL:
#
# - Get the average heart rate in the last X minutes available
#
# USAGE:
#
# - Call the function below with the duration of the period required in minutes
#
# - It is recommended to monitor last_sample_time to see if the data is up-to-date or not
#
###########################################################

def average_last_X_minutes_available(minutes):
  headers = {'Authorization': f'Bearer {access_token}'}

  response = requests.get(url, headers=headers)
  data = response.json()

  if response.status_code == 200:
    #  pprint.pprint(data['activities-heart-intraday']['dataset'][-1:])
    df = pd.DataFrame(data['activities-heart-intraday']['dataset'])

    # Parse the dates into datetime objectsd
    time_format = "%H:%M:%S"  # Specify the format matching YY:MM:DD
    df['parsed_time']=[datetime.strptime(time_string,time_format) for time_string in df['time']]

    last_sample_time = df['time'].iloc[-1]

    X_minutes_before_parsed_time = df['parsed_time'].iloc[-1] - timedelta(minutes=minutes)

    first_sample_parsed_time = next((time_parsed for time_parsed in df['parsed_time'] if time_parsed >= X_minutes_before_parsed_time), None)

    first_sample_time = first_sample_parsed_time.strftime("%H:%M:%S")

    return calculate_average_heart_rate(first_sample_time, last_sample_time), first_sample_time, last_sample_time

  else:
    pprint.pprint(data)

Tue Nov  5 01:12:20 PM AEDT 2024


In [52]:
# Example usage - calculate average in certain time range

rate = calculate_average_heart_rate('09:50:00','09:51:00')

print(rate)

DEBUG INFORMATION: 13 time stamps averaged:
DEBUG INFORMATION: ['09:50:00', '09:50:05', '09:50:10', '09:50:15', '09:50:20', '09:50:25', '09:50:30', '09:50:35', '09:50:40', '09:50:45', '09:50:50', '09:50:55', '09:51:00']
DEBUG INFORMATION: [125, 125, 126, 124, 122, 122, 118, 118, 121, 123, 120, 115, 110]
120.6923076923077


In [56]:
# Example usage - every minute, calculate the average in the last 5 minutes of available data

minutes = 5

while True:
  average_heart_rate, first_sample_time, last_sample_time = average_last_X_minutes_available(minutes)
  print(f"Last available time stamp: {last_sample_time}")
  print(f"Time stamp {minutes} minutes beforehand: {first_sample_time}")
  print(f"Average heart rate in this period: {average_heart_rate}")
  print("")
  time.sleep(60)

DEBUG INFORMATION: 45 time stamps averaged:
DEBUG INFORMATION: ['13:07:52', '13:08:07', '13:08:17', '13:08:22', '13:08:32', '13:08:37', '13:08:42', '13:08:47', '13:08:57', '13:09:02', '13:09:07', '13:09:12', '13:09:22', '13:09:37', '13:09:52', '13:09:57', '13:10:02', '13:10:12', '13:10:17', '13:10:22', '13:10:27', '13:10:32', '13:10:42', '13:10:47', '13:10:52', '13:11:02', '13:11:07', '13:11:12', '13:11:17', '13:11:22', '13:11:27', '13:11:32', '13:11:37', '13:11:42', '13:11:47', '13:12:02', '13:12:07', '13:12:12', '13:12:22', '13:12:27', '13:12:32', '13:12:37', '13:12:42', '13:12:47', '13:12:52']
DEBUG INFORMATION: [106, 104, 105, 106, 104, 103, 104, 105, 103, 101, 98, 94, 93, 92, 93, 94, 93, 94, 93, 92, 91, 91, 93, 94, 95, 97, 96, 95, 96, 97, 96, 92, 90, 91, 92, 92, 93, 94, 90, 89, 88, 87, 88, 91, 90]
Last available time stamp: 13:12:52
Time stamp 5 minutes beforehand: 13:07:52
Average heart rate in this period: 95.22222222222223

DEBUG INFORMATION: 45 time stamps averaged:
DEBUG INFO

KeyboardInterrupt: 